In [ ]:
!pip install jira
!pip install google-cloud-bigquery
!pip install google-cloud-storage

# Task
Create a link between Jira and BigQuery by extracting data from Jira and loading it into BigQuery.

## Setup jira connection

### Subtask:
Establish a connection to your Jira instance using appropriate libraries and credentials.


**Reasoning**:
Import the necessary library, define credentials, create a Jira client instance, and verify the connection by getting server information.



In [ ]:
from jira import JIRA
import os
from google.colab import userdata


# Define your Jira URL and credentials
jira_url = "https://facilitron.atlassian.net"
# jira_username = "bseeley@facilitron.com"
jira_username = 'ehan@facilitron.com'
jira_api_token = userdata.get('jira_api_token')
print(jira_api_token)

# Create an instance of the Jira client
jira = JIRA(server=jira_url, basic_auth=(jira_username, jira_api_token))

# Verify the connection by getting server information
server_info = jira.server_info()
print("Jira Server Information:")
print(f"  Version: {server_info['version']}")
print(f"  Base URL: {server_info['baseUrl']}")

Jira Server Information:
  Version: 1001.0.0-SNAPSHOT
  Base URL: https://facilitron.atlassian.net


In [ ]:
# Define the project key and the maximum number of results to retrieve
project_key = 'WORKS'  # Replace with your project key
max_results = 100  # Adjust as needed

# Search for issues in the specified project
issues = jira.search_issues(f'project={project_key}')

# Print the summary of the retrieved issues
print(f"Found {len(issues)} issues in project {project_key}:")
for issue in issues:
    print(f"- {issue.key}: {issue.fields.summary}")

Found 50 issues in project WORKS:
- WORKS-3703: Spinner Endlessly Loading - Fix
- WORKS-3702: Scheduler API: Validate Role & Pass Screen Parameter
- WORKS-3701: Advance Search -> Location causes errors when searching for multiple words
- WORKS-3700: Update status schema to remove reserved statuses
- WORKS-3699: Remove reserved statuses
- WORKS-3698: Create beamer post for remove reserved statuses
- WORKS-3697: Cost tab screen is no longer retrieving the correct hourly labor rate, it is returning $0 for all wages.
- WORKS-3696: Work order edit modal enhancements
- WORKS-3695: Optimize Order List Retrieval with Lazy Loading
- WORKS-3694: Misc Design Updates
- WORKS-3693: String was not recognized as a valid Boolean error in Elmah
- WORKS-3692: Fix chat window causing "changes not be saved" dialog
- WORKS-3691: Added Story for the Email alert 
- WORKS-3690: Add back date parameters to tom_GetActiveWorkOrdersForProcessorPaginationV4
- WORKS-3689: Increase dispositions to query with, Closed

**Reasoning**:
Authenticate to Google Cloud, create a BigQuery client instance, and define your project ID and dataset ID.

## Setup BigQuery Connection

### Subtask:
Establish a connection to your Google Cloud project and BigQuery dataset.

In [ ]:
from google.cloud import bigquery
from google.colab import auth

# Authenticate to Google Cloud
auth.authenticate_user()

# Define your Google Cloud project ID and BigQuery dataset ID
project_id = 'steady-cascade-427621' # Replace with your project ID
dataset_id = 'jira_imports' # Replace with your dataset ID

# Create a BigQuery client instance
client = bigquery.Client(project=project_id)

print(f"BigQuery client created for project: {project_id}")

BigQuery client created for project: steady-cascade-427621


In [ ]:
from datetime import datetime

# Define the table ID
table_id = f"{project_id}.{dataset_id}.works_issues" # Replace with your table ID

# Prepare the data for BigQuery
rows_to_insert = []
for issue in issues:
    # Parse and format timestamps
    created_timestamp = datetime.fromisoformat(issue.fields.created.replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    updated_timestamp = datetime.fromisoformat(issue.fields.updated.replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]

    rows_to_insert.append({
        "key": issue.key,
        "summary": issue.fields.summary,
        "description": issue.fields.description,
        "status": issue.fields.status.name,
        "created": created_timestamp,
        "updated": updated_timestamp,
        "priority": issue.fields.priority.name if issue.fields.priority else None,
        "issue_type": issue.fields.issuetype.name,
        "reporter_displayName": issue.fields.reporter.displayName if issue.fields.reporter else None,
        "assignee_displayName": issue.fields.assignee.displayName if issue.fields.assignee else None,
    })

# Define the table schema based on the selected fields
schema = [
    bigquery.SchemaField("key", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("summary", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("description", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("status", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("created", "TIMESTAMP", mode="REQUIRED"),
    bigquery.SchemaField("updated", "TIMESTAMP", mode="REQUIRED"),
    bigquery.SchemaField("priority", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("issue_type", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("reporter_displayName", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("assignee_displayName", "STRING", mode="NULLABLE"),
]

# Get the table reference
table_ref = client.dataset(dataset_id).table("works_issues")

# Check if the table exists, if not, create it
try:
    client.get_table(table_ref)  # API request
    print(f"Table {table_id} already exists.")
except Exception:
    print(f"Table {table_id} not found, creating table.")
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)  # API request
    print(f"Created table {table.project}.{table.dataset_id}.{table.table_id}")


# Load data into BigQuery
errors = client.insert_rows_json(table_ref, rows_to_insert)

if errors:
    print("Errors while inserting rows:")
    for error in errors:
        print(error)
else:
    print("Data loaded successfully into BigQuery.")

Table steady-cascade-427621.jira_imports.works_issues already exists.
Data loaded successfully into BigQuery.


In [ ]:
issues_with_changelog = []
for issue in issues:
    issue_with_changelog = jira.issue(issue.key, expand='changelog')
    issues_with_changelog.append(issue_with_changelog)

print(f"Fetched changelogs for {len(issues_with_changelog)} issues.")

Fetched changelogs for 50 issues.


In [ ]:
from datetime import datetime

changelog_rows_to_insert = []

for issue in issues_with_changelog:
    if issue.changelog and issue.changelog.histories:
        for history in issue.changelog.histories:
            created_timestamp = datetime.fromisoformat(history.created.replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
            for item in history.items:
                changelog_rows_to_insert.append({
                    "issue_key": issue.key,
                    "author_displayName": history.author.displayName if history.author else None,
                    "created": created_timestamp,
                    "field": item.field,
                    "fieldtype": item.fieldtype,

                    "from_string": item.fromString,

                    "to_string": item.toString,
                })

print(f"Processed {len(changelog_rows_to_insert)} changelog entries.")

Processed 430 changelog entries.


In [ ]:
changelog_schema = [
    bigquery.SchemaField("issue_key", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("author_displayName", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("created", "TIMESTAMP", mode="REQUIRED"),
    bigquery.SchemaField("field", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("fieldtype", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("from_value", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("from_string", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("to_value", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("to_string", "STRING", mode="NULLABLE"),
]

In [ ]:
# Define the changelog table ID
changelog_table_id = f"{project_id}.{dataset_id}.works_changelog"

# Get the changelog table reference
changelog_table_ref = client.dataset(dataset_id).table("works_changelog")

# Check if the changelog table exists, if not, create it
try:
    client.get_table(changelog_table_ref)  # API request
    print(f"Changelog table {changelog_table_id} already exists.")
except Exception:
    print(f"Changelog table {changelog_table_id} not found, creating table.")
    changelog_table = bigquery.Table(changelog_table_ref, schema=changelog_schema)
    changelog_table = client.create_table(changelog_table)  # API request
    print(f"Created changelog table {changelog_table.project}.{changelog_table.dataset_id}.{changelog_table.table_id}")


# Load changelog data into BigQuery
changelog_errors = client.insert_rows_json(changelog_table_ref, changelog_rows_to_insert)

if changelog_errors:
    print("Errors while inserting changelog rows:")
    for error in changelog_errors:
        print(error)
else:
    print("Changelog data loaded successfully into BigQuery.")

Changelog table steady-cascade-427621.jira_imports.works_changelog not found, creating table.
Created changelog table steady-cascade-427621.jira_imports.works_changelog
Changelog data loaded successfully into BigQuery.


## Load Changelog Data to BigQuery

### Subtask:
Load the transformed changelog data into the new BigQuery table.

**Reasoning**:
Define the schema for the BigQuery table based on the fields extracted from the changelog data.

## Define BigQuery Changelog Table Schema

### Subtask:
Define the schema for the new BigQuery table that will store the changelog data.

**Reasoning**:
Initialize an empty list, iterate through the issues with changelogs, iterate through the changelog entries for each issue, extract and format the relevant fields for each changelog item, and append the flattened data to the list.

## Transform changelog data

### Subtask:
Process the extracted changelog data to flatten and structure it for BigQuery.

## Load Data to BigQuery

### Subtask:
Load the extracted Jira data into your specified BigQuery table.

# Task
Import Jira issues and their changelogs into separate BigQuery tables. Modify the timestamp format of the Jira issue data to 'YYYY-MM-DD HH:MM[:SS[.SSSSSS]]' to resolve import errors.

## Extract jira changelogs

### Subtask:
Fetch the changelog for each issue from Jira using the Jira API.


**Reasoning**:
Iterate through the fetched issues and retrieve the changelog for each issue using the Jira API.



## Transform changelog data

### Subtask:
Process the extracted changelog data to flatten and structure it for BigQuery.


**Reasoning**:
Initialize an empty list, iterate through the issues with changelogs, iterate through the changelog entries for each issue, extract and format the relevant fields for each changelog item, and append the flattened data to the list.



**Reasoning**:
The previous code failed because the attribute `_from` does not exist in the `item` object. I need to access the `from` attribute directly. I will regenerate the full code block with this correction.



**Reasoning**:
The previous code failed again because the attribute `From` also does not exist. The correct attribute for the 'from' value in the changelog item should be `item.from_`. I will regenerate the full code block with this correction.



**Reasoning**:
The previous attempts to access the 'from' attribute of the changelog item have failed. I need to inspect the `item` object to understand its structure and identify the correct attribute name for the 'from' value. I will print the `item` object to examine its attributes.

